In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '..')

from pairing import Reader, Extractor
import definition
import numpy as np
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score

## Data Loading and Preprocessing

In [2]:
train_ratio = 0.6

raw_data = Reader.read_file(definition.DATA_PAIRED_SAMPLE)
index = np.arange(len(raw_data))
np.random.shuffle(index)
index_train = index[:int(train_ratio * len(raw_data))]
index_test = index[int(train_ratio * len(raw_data)):]
raw_data_train = np.array(raw_data)[index_train].tolist()
raw_data_test = np.array(raw_data)[index_test].tolist()

data_train = Extractor.extract_data(raw_data_train)
data_test = Extractor.extract_data(raw_data_test)

100%|███████████████████████████████████████████████████████████████████████████| 1080/1080 [00:00<00:00, 40006.43it/s]


In [3]:
data_train.head()

,dist_endpoint,dist_start,len_aspect_char,len_aspect_word,len_sentiment_char,len_sentiment_word,position_aspect,position_sentiment,target
0,1,2,3,1,8,2,2,0,1
1,8,8,3,1,5,1,2,10,0
2,6,7,15,3,8,2,7,0,0
3,1,3,15,3,5,1,7,10,1
4,1,2,7,2,4,1,0,2,1


In [4]:
data_train.describe()

,dist_endpoint,dist_start,len_aspect_char,len_aspect_word,len_sentiment_char,len_sentiment_word,position_aspect,position_sentiment,target
count,10759.000000,10759.000000,10759.000000,10759.000000,10759.000000,10759.000000,10759.000000,10759.000000,10759.000000
mean,7.758249,8.115066,7.913933,1.336741,8.235431,1.501627,10.427921,11.414816,0.325030
std,8.855695,8.890597,3.590752,0.591426,4.288563,0.701082,12.029538,12.323193,0.468408
min,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,5.000000,1.000000,5.000000,1.000000,2.000000,3.000000,0.000000
50%,5.000000,5.000000,8.000000,1.000000,7.000000,1.000000,6.000000,7.000000,0.000000
75%,10.000000,11.000000,10.000000,2.000000,11.000000,2.000000,14.000000,16.000000,1.000000
max,87.000000,88.000000,34.000000,6.000000,33.000000,6.000000,68.000000,88.000000,1.000000


In [5]:
X_train = data_train.drop('target', axis=1)
X_test = data_test.drop('target', axis=1)
y_train = data_train['target']
y_test = data_test['target']

## Training

In [6]:
model = LGBMClassifier()

In [7]:
model.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [8]:
pred = model.predict(X_test)

In [9]:
f1_score(y_test, pred)

0.8662940670679278